In [1]:
%env CUDA_VISIBLE_DEVICES=6
%env OMP_NUM_THREADS=2

env: CUDA_VISIBLE_DEVICES=6
env: OMP_NUM_THREADS=2


In [2]:
import sys
import os

sys.path.extend(['/home/jorg/repo/aesr'])
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import cm

import numpy as np
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import SimpleITK as sitk
try:
    from skimage.measure import compare_ssim as ssim_metric
    from skimage.measure import compare_psnr as psnr_metric
except ImportError:
    from skimage.metrics import structural_similarity as ssim_metric
    from skimage.metrics import peak_signal_noise_ratio as psnr_metric

from datasets.data_config import get_config
from datasets.shared_transforms import CenterCrop, AdjustToPatchSize
from kwatsch.get_trainer import get_trainer
from evaluate.create_HR_images import create_hr_images
from evaluate.brain.evaluate_model import create_brain_volumes
from evaluate.brain.evaluate_model import create_brain_volumes_conventional_interpolation
from kwatsch.common import load_settings
%matplotlib inline
%load_ext autoreload
%autoreload 2

torch.manual_seed(32563)
rs = np.random.RandomState(32563)
ttest_results = {}

print("Ready")

Ready


In [27]:
from datasets.MNIST.data3d import get_mnist_ids
pat_nums_test = get_mnist_ids('test')
print("#Volumes {}".format(len(pat_nums_test)))

INFO - get_mnist_ids - test
#Volumes 100


In [90]:
dataset = 'MNISTRoto' 
eval_patch_size = {'MNISTRoto': 32, 'MNIST3D': 28}[dataset]
model_nbr = 100 #  combined: dHCP: 1272  OASIS: 3mm 1298  OASIS: 2mm 507
model_nbr_sr =  None  # 836
output_dir = None
eval_axis = 0
model_name = 'acai'   # ae_combined    vae   acai
exper_dir = "pool2_l16_lamb05_noaug_ex01"
exper_src_path = "~/expers/sr_redo/{}/{}/{}".format(dataset, model_name, exper_dir)
myargs = load_settings(os.path.join(os.path.expanduser(exper_src_path), "settings.yaml"))
save_volumes, do_save_metrics = True, True
generate_inbetween_slices = True
use_original_slice = False
downsample_steps = 4  # myargs['downsample_steps']
num_interpolations = downsample_steps - 1
# output_dir =  exper_src_path + "/ni{:02d}".format(num_interpolations)
if not generate_inbetween_slices:
    num_interpolations = 6
    # IMPORTANT FOR BRAIN MRI WE USE THE DOWNSAMPLED VOLUMES EVEN WHEN USING HIGHER 
    # UPSAMPLING FACTORS THAN DURING TRAINING !!!
    downsample_steps = myargs['downsample_steps']
    
print("Eval patch size ", eval_patch_size)
metrics_dict = create_brain_volumes(exper_src_path, model_nbr, pat_list=pat_nums_test, 
                                   generate_inbetween_slices=generate_inbetween_slices,
                                   downsample_steps=downsample_steps, 
                                    save_volumes=save_volumes,
                                    eval_patch_size=eval_patch_size,
                                   num_interpolations=num_interpolations,
                                   compute_percept_loss=False, 
                                   use_original_slice=use_original_slice,
                                   eval_axis=eval_axis, model_nbr_sr=model_nbr_sr,
                                   output_dir=output_dir, do_save_metrics=do_save_metrics)

Eval patch size  32
INFO - Initializing trainer ACAITrainer - using multiple gpus False
INFO - ACAITrainer Loaded model parameters from /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/models/100.models
INFO - Trainer ACAI - Initiated discriminator
IMPORTANT --> ACAITrainer training with combined losses: False
WARNING WARNING - you are using VanillaACAI CAE class


Loading 100 volumes from /home/jorg/data/MNIST3D:   0%|          | 0/100 [00:00<?, ?it/s]

[36. 28. 28.]
WARNING - Current model-nbr >>> 100 <<< train patch-size/test patch-size 32/32
INFO - evaluating on 100 patients
WARNING !!! downsample_steps 3 does not match parameter 4 used during training!
{'ae_class': 'VanillaACAI', 'alpha_class': None, 'alpha_loss_func': None, 'aug_patch_size': 32, 'batch_size': 32, 'colors': 1, 'comment': None, 'dataset': 'MNISTRoto', 'depth': 16, 'device': 'cuda', 'downsample_steps': 3, 'epoch_threshold': 10, 'epochs': 100, 'ex_loss_weight1': 0.001, 'exper_id': 'pool2_l16_lamb05_noaug_ex01', 'fine_tune': False, 'get_masks': False, 'gpu_ids': [0], 'image_mix_loss_func': None, 'lamb': 0, 'lamb_reg_acai': 0.5, 'latent': 16, 'latent_width': 8, 'limited_load': False, 'log_tensorboard': False, 'lr': 1e-05, 'max_grad_norm': 0, 'model': 'acai', 'model_filename': None, 'module_network_path': 'networks/acai_vanilla.py', 'module_trainer_path': 'kwatsch/trainer_acai.py', 'n_res_block': None, 'number_of_workers': 2, 'output_dir': '/home/jorg/expers/sr_redo/MNI

SSIM / PSRN / VIF: 0.792 / 22.234 / 0.890
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60043/60043_ni03.nii.gz
SSIM / PSRN / VIF: 0.782 / 21.388 / 0.886
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60044/60044_ni03.nii.gz
SSIM / PSRN / VIF: 0.694 / 20.697 / 0.863
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60045/60045_ni03.nii.gz
SSIM / PSRN / VIF: 0.834 / 23.517 / 0.894
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60046/60046_ni03.nii.gz
SSIM / PSRN / VIF: 0.841 / 22.340 / 0.891
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60047/60047_ni03.nii.gz
SSIM / PSRN / VIF: 0.816 / 23.400 / 0.867
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60048/60048_ni03.nii.gz
SSIM / PSRN / VIF: 0.738 / 20.901 

SSIM / PSRN / VIF: 0.883 / 24.005 / 0.890
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60095/60095_ni03.nii.gz
SSIM / PSRN / VIF: 0.840 / 23.480 / 0.875
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60096/60096_ni03.nii.gz
SSIM / PSRN / VIF: 0.796 / 19.730 / 0.901
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60097/60097_ni03.nii.gz
SSIM / PSRN / VIF: 0.790 / 23.156 / 0.881
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60098/60098_ni03.nii.gz
SSIM / PSRN / VIF: 0.721 / 19.886 / 0.889
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60099/60099_ni03.nii.gz
SSIM / PSRN / VIF: 0.851 / 22.459 / 0.900
INFO - saved /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/images_sr_ip/60100/60100_ni03.nii.gz
SSIM / PSRN / VIF: 0.868 / 22.588 

In [11]:
from evaluate.common import save_metrics
print(metrics_dict.keys())
save_metrics(os.path.join(exper_src_path, 'results'), myargs['dataset'], metrics_dict, 
             downsample_steps,
                     'model{:04d}'.format(model_nbr), eval_axis)

dict_keys(['ssim', 'psnr', 'vif', 'lpips', 'ssim_synth', 'psnr_synth', 'vif_synth', 'lpips_synth', 'ssim_recon', 'psnr_recon', 'vif_recon', 'lpips_recon'])
INFO - Saved results to ~/expers/sr_redo/MNISTRoto/acai_combined/pool2_w28_l16_w1000_ex01/results/results/MNISTRoto_model0093_3x.npz


In [88]:
from evaluate.find_best_model import load_model_scores

file_suffix = "_axis0.npz"
synthesis = True
method = 'acai'
exper_dir = "~/expers/sr_redo/MNISTRoto/{}/pool2_l16_lamb05_noaug_ex01".format(method)
results, iters, ssim_res, psnr_res, vif_res = \
            load_model_scores(exper_dir, file_suffix=file_suffix, 
                              synthesis=synthesis)

INFO - searching with mask /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/model_perf_synth*_axis0.npz
INFO - loading /home/jorg/expers/sr_redo/MNISTRoto/acai/pool2_l16_lamb05_noaug_ex01/model_perf_synth_15_to_100_axis0.npz


In [89]:
idx_psnr, idx_ssim, idx_vif = np.argmax(psnr_res), np.argmax(ssim_res), np.argmax(vif_res)
print("SSIM ({}): {:.3f}".format(iters[idx_ssim], ssim_res[idx_ssim]))
print("PSNR ({}): {:.3f}".format(iters[idx_psnr], psnr_res[idx_psnr]))
print("VIF ({}): {:.3f}".format(iters[idx_vif], vif_res[idx_vif]))
print("M-{}: {:.3f}, {:.3f}, {:.3f}".format(iters[idx_ssim], ssim_res[idx_ssim],
                                          psnr_res[idx_ssim], vif_res[idx_ssim]))
print("M-{}: {:.3f}, {:.3f}, {:.3f}".format(iters[idx_psnr], ssim_res[idx_psnr],
                                          psnr_res[idx_psnr], vif_res[idx_psnr]))
print("M-{}: {:.3f}, {:.3f}, {:.3f}".format(iters[idx_vif], ssim_res[idx_vif],
                                          psnr_res[idx_vif], vif_res[idx_vif]))

SSIM (86): 0.835
PSNR (100): 19.994
VIF (26): 0.885
M-86: 0.835, 19.974, 0.882
M-100: 0.835, 19.994, 0.883
M-26: 0.818, 19.747, 0.885


In [84]:
from datasets.MNIST.data3d import get_mnist_ids
from datasets.common import get_images_in_dir
from datasets.common_brains import get_images
from collections import defaultdict
transform32 = transforms.Compose([AdjustToPatchSize(tuple((32, 32)))])

dataset = 'MNISTRoto'
patid_list = get_mnist_ids("test")
method_path, result_dict = {}, {}
data_conv = defaultdict(dict)
downsample_steps = 4
file_suffix = "_ni0{}.nii.gz".format(downsample_steps -1)
EXPER_SRC_DIR = os.path.expanduser("~/expers/sr_redo/{}".format(dataset))
image_ref = get_images(patid_list, 'MNIST3D', rescale_int=False, int_perc=tuple((0, 100)), 
                       limited_load=False, do_downsample=False, downsample_steps=downsample_steps, 
                       transform=transform32, include_hr_images=True)

INFO - get_mnist_ids - test


Loading 100 volumes from /home/jorg/data/MNIST3D:   0%|          | 0/100 [00:00<?, ?it/s]

[36. 32. 32.]


In [85]:
m_desc = 'vae2'
exper_id = {4: "{}/pool2_l16_beta1_noaug_ex01".format(m_desc)}[downsample_steps] 
file_suffix = "_ni0{}.nii.gz".format(downsample_steps -1)

path_method = os.path.join(EXPER_SRC_DIR, "{}/images_sr_ip".format(exper_id))
data_generator = get_images_in_dir(path_method, file_suffix=file_suffix, 
                                      rescale_int=False,  dataset_name=dataset,
                                          do_downsample=False,
                                          downsample_steps=None, patid_list=None)
print("Loaded from {} volumes from {} ({})".format(len(data_generator), exper_id,
                                                   file_suffix))
method_path[m_desc] = os.path.join(EXPER_SRC_DIR, "{}".format(exper_id))

INFO - get_images_in_dir - dataset name MNISTRoto (#100). Resample-inplane False
Loaded from 100 volumes from vae2/pool2_l16_beta1_noaug_ex01 (_ni03.nii.gz)


In [87]:
from evaluate.quantitative_comparison import compare_quantitative

method = 'vae2'
do_save = True
result_dict[method] = compare_quantitative(image_ref, data_generator, 
                                           method, downsample_steps=downsample_steps, 
                     is_4d=False, percept_criterion=None, do_save=do_save,
                                           eval_axis=0, transform=None,
                     method_dir=method_path[method])

INFO - evaluating vae2 images from /home/jorg/expers/sr_redo/MNISTRoto/vae2/pool2_l16_beta1_noaug_ex01
INFO - evaluation axis-0 - method: vae2 - is conventional False - downsample steps 4


Compute metrics:   0%|          | 0/100 [00:00<?, ?it/s]

vae2: SSIM / PSRN / LPIPS / VIF: 0.867 (0.06) / 21.17 (2.31) / 0.000 (0.00) / 0.884 (0.02)
vae2 (recon): SSIM / PSRN / LPIPS / VIF: 0.941 (0.04) / 24.56 (3.21) / 0.000 (0.00) / 0.911 (0.02)
vae2 (synth): SSIM / PSRN / LPIPS / VIF: 0.839 (0.07) / 19.89 (2.13) / 0.000 (0.00) / 0.874 (0.02)
INFO - Saved results to /home/jorg/expers/sr_redo/MNISTRoto/vae2/pool2_l16_beta1_noaug_ex01/results/vae2_4x.npz
